In [2]:
pip install twython

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\alana\anaconda3\python.exe -m pip install --upgrade pip' command.


In [21]:
import io, json, time, os, logging, argparse, atexit, gzip, sys
from tempfile import NamedTemporaryFile
from twython import Twython
from twython.exceptions import TwythonError, TwythonRateLimitError

In [22]:
# Minimal time accepted between two Rate Limit Errors
TOO_SOON = 10
# Time to wait if we receive a Rate Limit Error too soon after a previous one
WAIT_SOME_MORE = 60

SLEEP_SECS_DEFAULT = 0
REQUEST_LIMIT_DEFAULT = 180
BATCH_TIME_DEFAULT = 15 * 60

In [23]:
logging.basicConfig(format='%(asctime)s %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [24]:
def check_negative(value):
    value = int(value)
    if value < 0:
        raise argparse.ArgumentTypeError('%s is an invalid positive int value' % value)
    return value

In [28]:
parser = argparse.ArgumentParser(description=__doc__)
parser.add_argument('-i', '--input', required=False, help='Tweets IDs file, first element of each line is an ID. '
                                                          'If not set, the standard input is read.')
parser.add_argument('-o', '--output', required=True,help='Tweets dataset in jsonl: a complete Twython tweet json per line. One can set the same ''output file for several runs of this script: already downloaded tweets '
'are not re-processed, new tweets are appended.')
parser.add_argument('-r', '--responses', required=False,
                    help='If set, it outputs a json map from the http response statuses to the tweet IDs.')
parser.add_argument('-s', '--sleep', required=False, default=SLEEP_SECS_DEFAULT, type=check_negative,
                    help='Time to wait between each request, in seconds.')
parser.add_argument('-l', '--limit', required=False, default=REQUEST_LIMIT_DEFAULT, type=check_negative,
                    help='Number of requests before stopping for wating the "totaltime" passed since the first request.')
parser.add_argument('-t', '--totaltime', required=False, default=BATCH_TIME_DEFAULT, type=check_negative,
                    help='Window time necessary for each batch of "limit" requests, in seconds.')
parser.add_argument('-c', '--compress', required=False, choices=['bz2', 'gzip'],
                    help='Chose the compression format for the out files, no extension is appended.')

parser.add_argument('--consumerkey', required=True, help='hdVjKTuvX2wiv98KgluM8cYFM')
parser.add_argument('--consumersecret', required=True, help='IjNR3hibJyiMiJaxF3Hs2b4qruoiRDoKdRBrjsM3PHoeSDTnkw')
parser.add_argument('--accesstoken', required=True, help='1518754632553558016-KjDUbOADS4a68j24BDcUV5Rp2M0p3K')
parser.add_argument('--accesssecret', required=True, help='ZV4ZcclknUvEguBugn8FNumim0Ed3qyuqxmuFFc7WaaVS')

parser.add_argument('--test', required=False, action='store_true', default=False,
                    help='If set, run some tests for this script. All other parameters are ignored, '
                         'except for the authentication codes.')
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [-i INPUT] -o OUTPUT [-r RESPONSES]
                             [-s SLEEP] [-l LIMIT] [-t TOTALTIME]
                             [-c {bz2,gzip}] [--test]
                             hdVjKTuvX2wiv98KgluM8cYFM
                             IjNR3hibJyiMiJaxF3Hs2b4qruoiRDoKdRBrjsM3PHoeSDTnkw
                             1518754632553558016-KjDUbOADS4a68j24BDcUV5Rp2M0p3K
                             ZV4ZcclknUvEguBugn8FNumim0Ed3qyuqxmuFFc7WaaVS
ipykernel_launcher.py: error: the following arguments are required: -o/--output, IjNR3hibJyiMiJaxF3Hs2b4qruoiRDoKdRBrjsM3PHoeSDTnkw, 1518754632553558016-KjDUbOADS4a68j24BDcUV5Rp2M0p3K, ZV4ZcclknUvEguBugn8FNumim0Ed3qyuqxmuFFc7WaaVS


SystemExit: 2